In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers, optimizers

def tau(s,a):
    if (s[0] and s[4]) == 0 : s=np.roll(s,a)
    return s

def rho(s):
    return ((s[0]==1)+2*(s[4]==1))    

def terminal_state(s):
    return (s[0]==1 or s[4]==1)    

gamma=0.5
invT = 1

In [ ]:
# the network
inputs = layers.Input(shape=(5,))
h = layers.Dense(10, activation='relu')(inputs)
outputs = layers.Dense(2, activation='linear')(h)

model = models.Model(inputs=inputs, outputs=outputs)
RMSprop = optimizers.RMSprop(lr=0.01)
model.compile(loss='mse', optimizer=RMSprop)

In [ ]:
for trial in range(400):
    s= np.array([0, 0, 1, 0, 0])
    for t in range(0,5):
        if terminal_state(s): break
        if trial > 30 and invT > 0.1: invT -= 0.001
        prediction=model.predict(s.reshape(1,5), steps=1, verbose=0)
        aidx=np.argmax(prediction)
        if np.random.rand() < invT : aidx=1-aidx
        a=2*aidx-1
        next_s = tau(s,a)
        if terminal_state(next_s): 
            y = rho(next_s)
        else:
            y = gamma*np.max(model.predict(next_s.reshape(1,5), steps=1, verbose=0))
        prediction[0,aidx]=y
        model.fit(s.reshape(1,5), prediction, epochs=1, verbose=0)
        s = np.copy(next_s) 

In [ ]:
policy = np.zeros(5)
Q=[]
s = np.array([1,0,0,0,0])
for i in range(0,5):
    Qs=model.predict(s.reshape(1,5), steps=1)
    Q.append(Qs)
    aidx=np.argmax(Qs)
    policy[i]=2*aidx-1
    s = np.roll(s,1)
print(np.transpose(Q))
print('policy:',np.transpose(policy))
